In [1]:
print("Hello")

Hello


# 📄 Q&A Over PDF Files

## 🧩 Introduction

In this project, we will build a **Question & Answer (Q&A) application** that can respond to questions based on the content of **PDF documents**.

### 🧠 How It Works
- We use a **Document Loader** to extract and structure text from PDF files into a format that can be processed by an LLM (Large Language Model).
- We then construct a **Retrieval-Augmented Generation (RAG)** pipeline, which allows the model to retrieve relevant information and generate accurate answers — including **citations from the source document** when needed.

> **Note**: This project uses a basic implementation. In future projects, we will explore more advanced techniques for solving similar problems.


In [2]:
import os

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


from dotenv import load_dotenv, find_dotenv

In [3]:
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ['GROQ_API_KEY']
hf_token_api = os.environ['HF_TOKEN']
pinecone_api_key = os.environ['PINECONE_API_KEY']

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = r"Data\Be_Good.pdf"

docs = PyPDFLoader(pdf_path).load()

print(len(docs))

11


## 🔍 RAG (Retrieval-Augmented Generation)

To implement RAG, we will follow a standard pipeline involving vector storage and text chunking.

### 🧱 Components Used:

- We will use **Chroma DB** as the **vector database** (also called a *vector store*).
- A **text splitter** will be used to divide the loaded PDF into smaller chunks. These chunks are easier for the LLM to process within its context window.
- Once split, the chunks are loaded into the vector store.

### 🔄 Retrieval

We can then create a **retriever** from the vector store, which will be used in our **RAG chain** to find the most relevant content when answering user questions.

---

> 💡 If you're using the pre-loaded Poetry shell, you don't need to manually install the required packages — they’re already included.


In [7]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq


In [ ]:
ccccc

c:\Users\Dan\Desktop\Python LLm Engineer\01 Advanced ChatBot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)

chunk_of_text = text_splitter.split_documents(docs)

In [10]:
vector_db = FAISS.from_documents(documents=chunk_of_text, embedding=model_embeddings)

In [11]:
retriever = vector_db.as_retriever(search_kwargs={'k':3})

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# System prompt template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n"
    "{context}"
)

messages = [
    ("system", system_prompt),
    ("human", "{input}")
]

# Define the full chat prompt
prompt = ChatPromptTemplate.from_messages(messages)

# Create the document QA chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Create the full RAG retrieval chain
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [14]:
result = rag_chain.invoke({"input" : "What is this article about ?"})
result["answer"]

'This article, "Be Good" by Paul Graham, discusses the principles of building a successful startup, focusing on creating something people want and not worrying too much about the business model initially. It explores the idea that this approach can lead to a charity-like mindset, where the goal is to help people, and provides examples of successful companies that have adopted this approach.'

In [15]:
result

{'input': 'What is this article about ?',
 'context': [Document(metadata={'source': 'Data\\Be_Good.pdf', 'page': 0}, page_content="Be Good - Essay by Paul Graham\nBe Good\nBe good\nApril 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we\nstarted Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We've\nlearned a lot since then, but if I were choosing now that's still\nthe one I'd pick.Another thing we tell founders is not to worry too much about the\nbusiness model, at least at first.  Not because making money is\nunimportant, but because it's so much easier than building something\ngreat.A couple weeks ago I realized that if you put those two ideas\ntogether, you get something surprising.  Make something people want.\nDon't worry too much about making money.  What you've got is a\ndescription of a charity.When you get an unexpected result like this, it could either be a\nbug or a new discovery.  Either b